<a href="https://colab.research.google.com/github/Manaswi3104/Plant_Disease_Detection/blob/main/Soyaleaf_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!rm -rf yolov5


/content


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

# Install required dependencies
!pip install -U -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17493, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17493 (delta 1), reused 0 (delta 0), pack-reused 17490 (from 3)
Receiving objects: 100% (17493/17493), 16.59 MiB | 17.84 MiB/s, done.
Resolving deltas: 100% (11992/11992), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.7 MB/s eta 0:00:

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sivm205/soybean-diseased-leaf-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.93G/1.93G [00:24<00:00, 84.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1


In [5]:
import os

# Replace 'path' with the actual variable name if needed
for root, dirs, files in os.walk(path):
    print("📁 Directory:", root)
    for name in files[:5]:  # print first 5 files per folder
        print("   📄", name)
    print("---")

📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1
---
📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1/brown_spot
   📄 bs (1).bmp
   📄 bs (40).bmp
   📄 bs (22).jpg
   📄 bs (3).jpg
   📄 bs (25).bmp
---
📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1/Yellow Mosaic
   📄 YM (73).jpg
   📄 YM (40).jpg
   📄 YM (64).jpg
   📄 YM (26).jpg
   📄 YM (35).jpg
---
📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1/bacterial_blight
   📄 bb (14).jpg
   📄 bb (7).bmp
   📄 bb (35).bmp
   📄 bb (14).bmp
   📄 bb (18).bmp
---
📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1/ferrugen
   📄 DSC_0015.jpg
   📄 DSC_0057.jpg
   📄 DSC_0039.jpg
   📄 DSC_0032.jpg
   📄 DSC_0013.jpg
---
📁 Directory: /root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1/crestamento
   📄 5.jpg


In [7]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load dataset
data_path = '/root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1'
dataset = datasets.ImageFolder(root=data_path, transform=transform)

# Split into train and test sets, and load into DataLoader
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
!pip install split-folders

In [9]:
import splitfolders

splitfolders.ratio(
    input='/root/.cache/kagglehub/datasets/sivm205/soybean-diseased-leaf-dataset/versions/1',
    output='/content/yolov5/soybean_cls_split',
    seed=42,
    ratio=(.8, .2)  # 80% train, 20% val
)


Copying files: 701 files [00:16, 42.09 files/s]


In [11]:
!zip -r soybean_cls_split.zip /content/yolov5/soybean_cls_split


  adding: content/yolov5/soybean_cls_split/ (stored 0%)
  adding: content/yolov5/soybean_cls_split/train/ (stored 0%)
  adding: content/yolov5/soybean_cls_split/train/brown_spot/ (stored 0%)
  adding: content/yolov5/soybean_cls_split/train/brown_spot/bs (1).bmp (deflated 68%)
  adding: content/yolov5/soybean_cls_split/train/brown_spot/bs (40).bmp (deflated 36%)
  adding: content/yolov5/soybean_cls_split/train/brown_spot/bs (3).jpg (deflated 1%)
  adding: content/yolov5/soybean_cls_split/train/brown_spot/bs (25).bmp (deflated 50%)
  adding: content/yolov5/soybean_cls_split/train/brown_spot/bs (47).bmp (deflated 27%)
  adding: content/yolov5/soybean_cls_split/train/brown_spot/bs (6).bmp (deflated 36%)
  adding: content/yolov5/soybean_cls_split/train/brown_spot/bs (11).bmp (deflated 63%)
  adding: content/yolov5/soybean_cls_split/train/brown_spot/bs (24).jpg (deflated 5%)
  adding: content/yolov5/soybean_cls_split/train/brown_spot/bs (30).bmp (deflated 68%)
  adding: content/yolov5/soybea

In [12]:
from google.colab import files
files.download("soybean_cls_split.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
%cd /content/yolov5

!python classify/train.py \
  --model yolov5s-cls.pt \
  --data soybean_cls_split \
  --epochs 20 \
  --img 224 \
  --batch 32

/content/yolov5
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-18 04:38:41.193947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been regi

In [13]:
!python classify/val.py \
  --weights runs/train-cls/exp/weights/best.pt \
  --data soybean_cls_split \
  --imgsz 224 \
  --batch-size 128 \
  --verbose



/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
classify/val: data=soybean_cls_split, weights=['runs/train-cls/exp/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4179498 parameters, 0 gradients, 10.4 GFLOPs
validating:   0% 0/2 [00:00<?, ?it/s]/content/yolov5/classify/val.py:111: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=device.type != "cpu"):
validating: 100% 2/2 [00:14<00:00, 

In [ ]:
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source /content/image.jpeg


/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/image.jpeg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4179498 parameters, 0 gradients, 10.4 GFLOPs
100% 755k/755k [00:00<00:00, 21.3MB/s]
image 1/1 /content/image.jpeg: 224x224 brown_spot 0.91, septoria 0.01, Southern blight 0.01, Yellow Mosaic 0.01, Mossaic Virus 0.01, 3.0ms
Speed: 0

In [14]:
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source /content/image4.jpeg

/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/image4.jpeg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4179498 parameters, 0 gradients, 10.4 GFLOPs
100% 755k/755k [00:00<00:00, 22.2MB/s]
image 1/1 /content/image4.jpeg: 224x224 Sudden Death Syndrone 0.90, ferrugen 0.02, Mossaic Virus 0.01, bacterial_blight 0.01, crestamento 0.01, 3.

In [ ]:
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source /content/image2.jpeg

/content/yolov5/utils/general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/image2.jpeg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-421-g79c4c31d Python-3.11.13 torch-2.7.1+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 117 layers, 4179498 parameters, 0 gradients, 10.4 GFLOPs
image 1/1 /content/image2.jpeg: 224x224 Sudden Death Syndrone 0.92, ferrugen 0.01, crestamento 0.01, septoria 0.01, brown_spot 0.01, 3.0ms
Speed: 0.3ms pre-process, 3.0ms inference, 13.

In [ ]:
# If in Google Colab
from google.colab import files
files.download('runs/train-cls/exp/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
!pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 9.5 MB/s eta 0:00:00


In [16]:
!pip install pyngrok


In [ ]:
!pip install pillow

In [17]:
import importlib
import subprocess
import sys

def install_if_missing(package):
    try:
        importlib.import_module(package)
        print(f"{package} is already installed ✅")
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Check and install each package
install_if_missing("streamlit")
install_if_missing("pyngrok")
install_if_missing("torch")
install_if_missing("torchvision")


streamlit is already installed ✅
pyngrok is already installed ✅
torch is already installed ✅
torchvision is already installed ✅


In [18]:
!curl ipv4.icanhazip.com

34.125.103.42


In [19]:
!ngrok config add-authtoken 2yZp8YUfuFWKVRJ6TiOkl84nvVa_5MQeNrFd95Preq7xtVHex


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [21]:
!pkill -f streamlit
!killall ngrok


In [ ]:
from torchvision.datasets import ImageFolder
dataset = ImageFolder("/content/yolov5/soybean_cls_split")
print(dataset.classes)

['train', 'val']


In [22]:
from pyngrok import ngrok
import threading
import time
import os

# Kill old streamlit/ngrok if running
os.system("pkill -f streamlit")
os.system("killall ngrok")

def run():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run)
thread.start()
time.sleep(5)

# Optional: set auth token (only once per Colab session)
# ngrok.set_auth_token("your_token_here")

public_url = ngrok.connect(8501)
print("App running at:", public_url)


App running at: NgrokTunnel: "https://f017-34-125-103-42.ngrok-free.app" -> "http://localhost:8501"
